# Lab 3: Chaining Prompts with LangChain

## Objectives
- Learn LangChain fundamentals
- Build multi-step prompt chains
- Implement memory in conversations
- Create custom tools and agents

## Prerequisites
- Completed Labs 1 and 2
- Understanding of prompt engineering concepts

In [ ]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import load_tools, initialize_agent, AgentType

load_dotenv()

# Initialize OpenAI LLM
llm = OpenAI(temperature=0.7)

## 1. Basic Prompt Chains

In [ ]:
# Create a chain for generating a story idea
story_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a short story idea about {topic}."
)

# Create a chain for generating a title
title_prompt = PromptTemplate(
    input_variables=["story"],
    template="Generate a creative title for this story: {story}"
)

# Create individual chains
story_chain = LLMChain(llm=llm, prompt=story_prompt)
title_chain = LLMChain(llm=llm, prompt=title_prompt)

# Combine chains
story_generation_chain = SimpleSequentialChain(
    chains=[story_chain, title_chain],
    verbose=True
)

# Test the chain
result = story_generation_chain.run("artificial intelligence")
print("Final Title:", result)

## 2. Implementing Conversation Memory

In [ ]:
class ConversationalAssistant:
    def __init__(self):
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "human_input"],
            template="""Previous conversation:
            {chat_history}
            
            Human: {human_input}
            Assistant: """
        )
        
        self.conversation = LLMChain(
            llm=llm,
            prompt=self.prompt,
            memory=self.memory,
            verbose=True
        )
    
    def chat(self, user_input: str) -> str:
        return self.conversation.predict(human_input=user_input)

# Test the conversational assistant
assistant = ConversationalAssistant()

# Have a multi-turn conversation
questions = [
    "What is machine learning?",
    "Can you give me an example?",
    "How is that different from deep learning?"
]

for question in questions:
    print(f"\nHuman: {question}")
    response = assistant.chat(question)
    print(f"Assistant: {response}")

## 3. Creating Custom Tools and Agents

In [ ]:
from langchain.tools import Tool
from typing import List

class CodeAssistant:
    def __init__(self):
        # Define custom tools
        self.tools = [
            Tool(
                name="code_generator",
                func=self._generate_code,
                description="Generates Python code based on a description"
            ),
            Tool(
                name="code_explainer",
                func=self._explain_code,
                description="Explains what a piece of code does"
            ),
            Tool(
                name="code_reviewer",
                func=self._review_code,
                description="Reviews code and suggests improvements"
            )
        ]
        
        # Initialize the agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
    
    def _generate_code(self, description: str) -> str:
        prompt = f"Generate Python code that does the following: {description}"
        return llm(prompt)
    
    def _explain_code(self, code: str) -> str:
        prompt = f"Explain what this code does in simple terms:\n{code}"
        return llm(prompt)
    
    def _review_code(self, code: str) -> str:
        prompt = f"Review this code and suggest improvements:\n{code}"
        return llm(prompt)
    
    def assist(self, query: str) -> str:
        return self.agent.run(query)

# Test the code assistant
assistant = CodeAssistant()

queries = [
    "Generate a Python function to calculate the fibonacci sequence",
    "Explain what this code does: def quicksort(arr): return arr if len(arr) <= 1 else quicksort([x for x in arr[1:] if x <= arr[0]]) + [arr[0]] + quicksort([x for x in arr[1:] if x > arr[0]])",
    "Review this code: def process(x): return x + 1"
]

for query in queries:
    print(f"\nQuery: {query}")
    response = assistant.assist(query)
    print(f"Response: {response}")

## 4. Building a Research Assistant

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.chains import RetrievalQA

class ResearchAssistant:
    def __init__(self):
        # Initialize search tool
        self.search = DuckDuckGoSearchRun()
        
        # Create tools list
        self.tools = [
            Tool(
                name="web_search",
                func=self.search.run,
                description="Useful for searching the web for current information"
            ),
            Tool(
                name="summarize",
                func=self._summarize_text,
                description="Summarizes a piece of text"
            )
        ]
        
        # Initialize the agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
    
    def _summarize_text(self, text: str) -> str:
        prompt = f"Summarize the following text:\n{text}"
        return llm(prompt)
    
    def research(self, query: str) -> str:
        return self.agent.run(query)

# Test the research assistant
researcher = ResearchAssistant()

research_queries = [
    "What are the latest developments in quantum computing?",
    "Summarize the key benefits of using LangChain for AI development"
]

for query in research_queries:
    print(f"\nResearch Query: {query}")
    response = researcher.research(query)
    print(f"Findings: {response}")

## Exercises

1. Create a multi-step chain for processing and analyzing text data
2. Build an agent that can interact with multiple APIs
3. Implement a document question-answering system using LangChain
4. Create a custom tool for specific domain tasks

## Next Steps
- Explore more advanced LangChain features
- Implement custom memory systems
- Create domain-specific agents
- Build more complex tool combinations